# 할 일

1. 구별 클러스터링
2. 클러스터별 면적 계산
3. 사고발생건수/면적 계산: 기준 이상만 남기기

In [62]:
import warnings
import pyproj
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium import plugins
from statsmodels.api import OLS
from sklearn.cluster import DBSCAN
from functools import partial
from shapely.ops import transform
from shapely.geometry import Point

In [2]:
warnings.filterwarnings(action="ignore")

In [3]:
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
def geodesic_point_buffer(point, meter):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=point.y, lon=point.x)),
        proj_wgs84
    )
    buf = Point(0, 0).buffer(meter)  # distance in metres
    return transform(project, buf)

In [4]:
def calc_meter_area(polygon):
    proj = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),
        pyproj.Proj(init='epsg:3857')
    )
    s_new = transform(proj, polygon)
    return s_new.area

In [5]:
data = pd.read_json("data/kids-accident-pp.json")
data = data.assign(
    sido = [x.split()[0] for x in data.legaldong_name],
    gugun = [x.split()[1] for x in data.legaldong_name],
    acdnt_dd_dc = pd.to_datetime(data.acdnt_dd_dc, format="%Y-%m-%d")
)

In [6]:
gdf = gpd.read_file("data/geometry.geojson")
gdf = gdf.assign(rad_X = np.radians(gdf.geometry.x), rad_Y=np.radians(gdf.geometry.y))

# 1. 기준 설정

In [7]:
acdnt_cnt = (data.groupby(data.acdnt_dd_dc.dt.year)
                 .size()
                 .reset_index()
                 .rename(columns={"acdnt_dd_dc":"year", 0:"cnt"})
                 .assign(constant=1)
            )

In [8]:
ols_model = OLS(acdnt_cnt.cnt.values,acdnt_cnt[["constant","year"]]).fit()
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     245.5
Date:                Mon, 15 Jun 2020   Prob (F-statistic):           7.20e-09
Time:                        13:45:52   Log-Likelihood:                -91.486
No. Observations:                  13   AIC:                             187.0
Df Residuals:                      11   BIC:                             188.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant    7.054e+05   4.47e+04     15.787      0.000    6.07e+05    8.04e+05
year        -347.7637     22.197    -15.667      0.000    -396.620    -298.908
==============================================================================
Omnibus:                        0.697   Durbin-Watson:                   0.847
Prob(Omnibus):                  0.706   Jarque-Bera (JB):                0.015
Skew:                          -0.076   Prob(JB):                        0.992
Kurtosis:                       3.071   Cond. No.                     1.08e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.08e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
summation = 2
criterion = 2
print("year","|","criterion")
print("====","|","="*20)
print(acdnt_cnt.year.values[-1],"|",criterion)
for i in range(acdnt_cnt.shape[0]-1):
    criterion = criterion/ols_model.fittedvalues.values[-(i+1)] * ols_model.fittedvalues.values[-(i+2)]
    summation += criterion
    print(acdnt_cnt.year.values[-(i+2)], "|", criterion)

year | criterion
==== | ====================
2019 | 2
2018 | 2.212058873785856
2017 | 2.4241177475717115
2016 | 2.636176621357638
2015 | 2.848235495143494
2014 | 3.0602943689293496
2013 | 3.272353242715205
2012 | 3.484412116501132
2011 | 3.6964709902869877
2010 | 3.9085298640728436
2009 | 4.120588737858699
2008 | 4.332647611644555
2007 | 4.544706485430481


In [10]:
# 2건 X 13년 = 26
criterion = 26 / (300 * 300 * np.pi)

# 2. 클러스터 선정

In [11]:
seoul = data.query("sido=='서울특별시'").drop("sido", axis=1)

In [12]:
earth_radius_km = 6371
epsilon = 0.15 / earth_radius_km #calculate 150 meter epsilon threshold
min_samples = int(26/4)

In [57]:
result = []
for gu in seoul.gugun.unique():
    print(gu)
    # 클러스터 모델 적합
    df = seoul[seoul.gugun==gu]
    geo = gdf[gdf.acdnt_no.isin(df.acdnt_no)].drop("acdnt_no",axis=1)
    model = DBSCAN(
        eps=epsilon,
        min_samples=min_samples,
        n_jobs=6
    )
    model.fit(geo[["rad_X", "rad_Y"]])
    if model.labels_.max()==-1:
        print("-----------------------------------")
        print("클러스터 없음")
        print("-----------------------------------\n")
        continue
    before = pd.Series(model.labels_).value_counts().sort_index().reset_index().rename(columns={"index":"cluster", 0:"count"})
    
    # 밀도 계산
    geo = geo.assign(
        labels = model.labels_
    ).query("labels!=-1").assign(geometry = geo.geometry.apply(geodesic_point_buffer, meter=100))
    
    geo = pd.merge(
        geo.dissolve(by="labels").reset_index(),
        geo.labels.value_counts().reset_index().rename(columns={"labels":"cls_size", "index":"labels"})
    )
    geo = geo.assign(meter_area = geo.geometry.apply(calc_meter_area))
    geo = geo.assign(density = geo.cls_size/geo.meter_area)

    print("-----------------------------------")
    print(
        before.assign(
            alive = before.cluster.isin(geo[geo.density > criterion].labels),
            density = [0] + list(geo.sort_values("labels").density)
        )
    )
    print("-----------------------------------\n")
    result.append(geo.copy())

도봉구
-----------------------------------
    cluster  count  alive   density
0        -1    209  False  0.000000
1         0     21  False  0.000066
2         1     23  False  0.000061
3         2     20  False  0.000070
4         3     14  False  0.000048
5         4     16  False  0.000068
6         5     10  False  0.000049
7         6      9  False  0.000086
8         7     15  False  0.000047
9         8     10  False  0.000090
10        9      8  False  0.000065
11       10      6  False  0.000045
12       11      6  False  0.000053
13       12     11  False  0.000059
14       13      9  False  0.000053
15       14      6  False  0.000052
16       15      6  False  0.000042
-----------------------------------

강남구
-----------------------------------
    cluster  count  alive   density
0        -1    352  False  0.000000
1         0      9  False  0.000052
2         1      8  False  0.000051
3         2     18  False  0.000054
4         3      8  False  0.000048
5         4      7 

-----------------------------------
   cluster  count  alive   density
0       -1    141  False  0.000000
1        0     19  False  0.000058
2        1     16  False  0.000059
3        2      9  False  0.000051
4        3      6  False  0.000061
5        4     10  False  0.000068
6        5      8  False  0.000057
7        6      6  False  0.000044
8        7      8  False  0.000050
-----------------------------------

성동구
-----------------------------------
    cluster  count  alive   density
0        -1    169  False  0.000000
1         0     16  False  0.000069
2         1     14  False  0.000059
3         2     13  False  0.000065
4         3     20  False  0.000050
5         4      8  False  0.000056
6         5      7  False  0.000043
7         6      7   True  0.000125
8         7      6  False  0.000053
9         8      6  False  0.000053
10        9      6  False  0.000039
11       10     11  False  0.000049
-----------------------------------

송파구
----------------------------

-----------------------------------
   cluster  count  alive   density
0       -1    142  False  0.000000
1        0      6  False  0.000047
2        1      6  False  0.000054
3        2     12  False  0.000062
4        3     10  False  0.000066
5        4      6  False  0.000061
6        5      6  False  0.000038
7        6      8  False  0.000048
-----------------------------------

강북구
-----------------------------------
    cluster  count  alive   density
0        -1    206  False  0.000000
1         0     14  False  0.000090
2         1     33  False  0.000078
3         2      8  False  0.000049
4         3      7  False  0.000081
5         4     10  False  0.000058
6         5      6  False  0.000052
7         6      7  False  0.000051
8         7     26  False  0.000071
9         8     26  False  0.000075
10        9     14  False  0.000056
11       10     14  False  0.000049
12       11     20  False  0.000072
13       12     12  False  0.000045
14       13      9  False  0.000

# 3. 지도 시각화

In [58]:
cluster = pd.concat(result).query(f"density > {criterion}")
acdnt_cls = gpd.sjoin(gdf[['geometry', 'acdnt_no']], cluster[['labels', 'geometry']],  op="within")
acdnt_cls = pd.merge(acdnt_cls, data)
acdnt_cls = acdnt_cls.assign(acdnt_dd_dc = acdnt_cls.acdnt_dd_dc.astype(str))

In [60]:
m = folium.Map(location=[37.53, 126.97], zoom_start=12) 

folium.features.GeoJson(cluster).add_to(m)
plugins.ScrollZoomToggler().add_to(m)
folium.features.GeoJson(acdnt_cls, piste_style_function).add_to(m)

In [61]:
m